In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib
import os

In [2]:
        
RDS_HOST   = os.getenv("SQL_SERVER_HOST")
RDS_PORT   = int(os.getenv("SQL_SERVER_PORT"))
RDS_DB     = os.getenv("SQL_SERVER_DATABASE")
RDS_USER   = os.getenv("SQL_SERVER_USER")
RDS_PASS   = urllib.parse.quote_plus(os.getenv("SQL_SERVER_PASSWORD"))

In [3]:
connection_string = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={RDS_HOST},{RDS_PORT};"
    f"DATABASE={RDS_DB};"
    f"UID={RDS_USER};PWD={RDS_PASS};"
    f"TrustServerCertificate=yes;"
)

engine = create_engine(
    f"mysql+mysqlconnector://{RDS_USER}:{RDS_PASS}@{RDS_HOST}:{RDS_PORT}/{RDS_DB}"
)

In [4]:

with engine.begin() as conn:
    # Temporarily disable foreign key checks
    conn.execute(text("SET foreign_key_checks = 0"))

    result = conn.execute(text("SHOW FULL TABLES WHERE Table_type = 'BASE TABLE'"))
    tables = result.fetchall()

    for table in tables:
        table_name = table[0]
        print(f"Dropping table: {table_name}")
        conn.execute(text(f"DROP TABLE `{table_name}`"))

    # Re-enable foreign key checks
    conn.execute(text("SET foreign_key_checks = 1"))

print("✅ All tables dropped successfully!")

✅ All tables dropped successfully!


In [5]:
from pathlib import Path          # ← you were missing this
import pandas as pd
from sqlalchemy import create_engine

def load_folder(folder_path: str, engine) -> None:
    """
    Load every CSV or Excel file in *folder_path* into the database.
    Table name = file name (without extension). Existing tables are replaced.
    """
    for file in Path(folder_path).iterdir():
        if file.suffix.lower() not in {".csv", ".xlsx"}:
            continue  # skip everything else

        table = file.stem.lower()                 # simple table name
        df = pd.read_csv(file) if file.suffix == ".csv" else pd.read_excel(file)

        df.head(0).to_sql(table, engine, if_exists="replace", index=False)
        df.to_sql(table, engine, if_exists="append",  index=False, method="multi", chunksize=1000)

        print(f"Loaded {file.name} → {table}")

# -------------- minimal demo --------------
if __name__ == "__main__":

    # call the helper
    load_folder("./data", engine)


Loaded order_items.csv → order_items
Loaded orders.csv → orders
Loaded product_variants.csv → product_variants
Loaded user_audit.csv → user_audit
Loaded support_messages.csv → support_messages
Loaded wishlist_items.csv → wishlist_items
Loaded products.csv → products
Loaded cities.csv → cities
Loaded business_representatives.csv → business_representatives
Loaded cart_items.csv → cart_items
Loaded stores.csv → stores
Loaded payments.csv → payments
Loaded order_status_history.csv → order_status_history
Loaded discounts.csv → discounts
Loaded product_tags.csv → product_tags
Loaded returns.csv → returns
Loaded carts.csv → carts
Loaded messages.csv → messages
Loaded wishlists.csv → wishlists
Loaded reviews.csv → reviews
Loaded shipping_info.csv → shipping_info
Loaded businesses.csv → businesses
Loaded tags.csv → tags
Loaded vendor_applications.csv → vendor_applications
Loaded support_tickets.csv → support_tickets
Loaded users.csv → users
Loaded address.csv → address
Loaded categories.csv → c